# In silico identification of switching nodes (ISIS), by F. Mairet 

## <I>Escherichia coli</I> (Core network) under aerobic vs anaerobic conditions

In [1]:
import cobra
import numpy as np
from pandas import DataFrame
import pandas as pd
model=cobra.io.read_sbml_model("e_coli_core.xml")

### Computation of fluxes with pFBA

In [2]:
# Aerobic condition
pfba1 = cobra.flux_analysis.pfba(model)
model.summary()

IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      21.8   h2o_e  29.2   BIOMASS_Ecol...  0.874
glc__D_e  10     co2_e  22.8
nh4_e      4.77  h_e    17.5
pi_e       3.21


In [3]:
# Anaerobic condition
med=model.medium
med["EX_o2_e"] = 0.0
model.medium=med
pfba2 = cobra.flux_analysis.pfba(model)
model.summary()

IN FLUXES         OUT FLUXES     OBJECTIVES
----------------  -------------  ----------------------
glc__D_e  10      h_e     30.6   BIOMASS_Ecol...  0.212
h2o_e      7.12   for_e   17.8
nh4_e      1.15   ac_e     8.5
pi_e       0.779  etoh_e   8.28
co2_e      0.378


### Identification of switch nodes

In [4]:
# Concatenation and normalization of fluxes
V=(np.concatenate((np.array(pfba1.fluxes)[np.newaxis,:],np.array(pfba2.fluxes)[np.newaxis,:]),axis=0)).T
V=np.divide(V,np.linalg.norm(V,axis=0))

df1=DataFrame(V,index=model.reactions,columns=['Aerobic','Anaerobic'])
df1.to_excel('Ecoli_core_Flux.xlsx', sheet_name='sheet1', index=True)

S=cobra.util.array.create_stoichiometric_matrix(model,array_type='dense') # stoichiometric matrix

nm=S.shape[0] # number of metabolites

# loop on all the metabolites to compute their score
r=np.zeros(nm)
for i in range (0,nm):
    jj=S[i,:]!=0
    M=np.multiply(S[i,jj][np.newaxis,:].T, V[jj,:]) # compute M, removing the zero rows 
    
    u, s, vh = np.linalg.svd(M)
    
    if s[0]==0:
        r[i]=0
    else:
        r[i]=1-pow(s[0],2)/sum(pow(s,2))
        
# Export and print the results
df2=pd.concat([DataFrame(model.metabolites, columns=['Metabolite']), DataFrame(r, columns=['score'])], axis = 1) 
df2.to_excel('Ecoli_core_Metabolite.xlsx', sheet_name='sheet1', index=False)

ind=np.argsort(r)
for i in range (1,nm):
    print(model.metabolites[ind[-i]],':',r[ind[-i]])

nadh_c : 0.3160085009094188
nad_c : 0.3160085009094188
adp_c : 0.21546825897196853
atp_c : 0.21546825897196853
pi_c : 0.18896744287602008
h2o_c : 0.15478220415037358
coa_c : 0.14486085172227514
nadph_c : 0.12289005151268739
nadp_c : 0.12289005151268739
pyr_c : 0.12060007143820894
accoa_c : 0.10892263563732696
h_c : 0.056699008211364865
h_e : 0.05592390537053393
g6p_c : 0.048549119700144194
akg_c : 0.024845335219471965
pep_c : 0.015857306181454867
oaa_c : 0.012844202869619958
f6p_c : 0.011925845808906654
r5p_c : 0.010539646090508614
e4p_c : 0.007991564014719788
g3p_c : 0.003078298323075268
ru5p__D_c : 0.0029538996962434894
co2_c : 0.001747009859031201
xu5p__D_c : 0.0009565913334195209
3pg_c : 0.0005877438989744244
13dpg_c : 0.0
o2_e : 0.0
2pg_c : 0.0
o2_c : 0.0
nh4_e : 0.0
glu__L_e : 0.0
nh4_c : 0.0
gln__L_c : 0.0
glx_c : 0.0
mal__L_e : 0.0
6pgc_c : 0.0
mal__L_c : 0.0
lac__D_e : 0.0
glu__L_c : 0.0
lac__D_c : 0.0
icit_c : 0.0
h2o_e : 0.0
gln__L_e : 0.0
q8_c : 0.0
6pgl_c : 0.0
pi_e : 0.0
